# مائیکروسافٹ ایجنٹ فریم ورک کے ساتھ انسانی مداخلت کا ورک فلو

## 🎯 سیکھنے کے مقاصد

اس نوٹ بک میں، آپ سیکھیں گے کہ مائیکروسافٹ ایجنٹ فریم ورک کے `RequestInfoExecutor` کا استعمال کرتے ہوئے **انسانی مداخلت** کے ورک فلو کو کیسے نافذ کیا جائے۔ یہ طاقتور طریقہ کار AI ورک فلو کو انسانی ان پٹ حاصل کرنے کے لیے روکنے کی اجازت دیتا ہے، جس سے آپ کے ایجنٹس انٹرایکٹو بن جاتے ہیں اور اہم فیصلوں پر انسانوں کو کنٹرول فراہم کرتے ہیں۔

## 🔄 انسانی مداخلت کیا ہے؟

**انسانی مداخلت (HITL)** ایک ڈیزائن پیٹرن ہے جہاں AI ایجنٹس عمل کو روک کر انسانی ان پٹ طلب کرتے ہیں اور پھر آگے بڑھتے ہیں۔ یہ درج ذیل کے لیے ضروری ہے:

- ✅ **اہم فیصلے** - اہم اقدامات کرنے سے پہلے انسانی منظوری حاصل کریں
- ✅ **غیر واضح حالات** - جب AI غیر یقینی ہو تو انسانوں کو وضاحت کرنے دیں
- ✅ **صارف کی ترجیحات** - صارفین سے متعدد اختیارات میں سے انتخاب کرنے کو کہیں
- ✅ **ضابطہ اور حفاظت** - ریگولیٹڈ آپریشنز کے لیے انسانی نگرانی کو یقینی بنائیں
- ✅ **انٹرایکٹو تجربات** - ایسے گفتگو کرنے والے ایجنٹس بنائیں جو صارف کے ان پٹ کا جواب دیں

## 🏗️ مائیکروسافٹ ایجنٹ فریم ورک میں یہ کیسے کام کرتا ہے

فریم ورک HITL کے لیے تین اہم اجزاء فراہم کرتا ہے:

1. **`RequestInfoExecutor`** - ایک خاص ایگزیکیوٹر جو ورک فلو کو روکتا ہے اور `RequestInfoEvent` کو خارج کرتا ہے
2. **`RequestInfoMessage`** - انسانوں کو بھیجے جانے والے ٹائپڈ درخواست پے لوڈز کے لیے بنیادی کلاس
3. **`RequestResponse`** - `request_id` کا استعمال کرتے ہوئے انسانی جوابات کو اصل درخواستوں کے ساتھ جوڑتا ہے

**ورک فلو پیٹرن:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 ہمارا مثال: صارف کی تصدیق کے ساتھ ہوٹل کی بکنگ

ہم مشروط ورک فلو پر کام کریں گے اور متبادل مقامات تجویز کرنے سے **پہلے** انسانی تصدیق شامل کریں گے:

1. صارف ایک مقام کی درخواست کرتا ہے (مثلاً، "پیرس")
2. `availability_agent` چیک کرتا ہے کہ آیا کمرے دستیاب ہیں
3. **اگر کمرے دستیاب نہیں ہیں** → `confirmation_agent` پوچھتا ہے "کیا آپ متبادل دیکھنا چاہتے ہیں؟"
4. ورک فلو **روکتا ہے** `RequestInfoExecutor` کا استعمال کرتے ہوئے
5. **انسان جواب دیتا ہے** "ہاں" یا "نہیں" کنسول ان پٹ کے ذریعے
6. `decision_manager` جواب کی بنیاد پر راستہ طے کرتا ہے:
   - **ہاں** → متبادل مقامات دکھائیں
   - **نہیں** → بکنگ کی درخواست منسوخ کریں
7. حتمی نتیجہ دکھائیں

یہ مظاہرہ کرتا ہے کہ صارفین کو ایجنٹ کی تجاویز پر کنٹرول کیسے دیا جا سکتا ہے!

---

چلیں شروع کرتے ہیں! 🚀


## مرحلہ 1: مطلوبہ لائبریریاں درآمد کریں

ہم معیاری ایجنٹ فریم ورک کے اجزاء کے ساتھ **انسانی مداخلت کے مخصوص کلاسز** درآمد کرتے ہیں:
- `RequestInfoExecutor` - ایسا ایگزیکیوٹر جو انسانی ان پٹ کے لیے ورک فلو کو روکتا ہے
- `RequestInfoEvent` - ایسا ایونٹ جو انسانی ان پٹ کی درخواست پر جاری ہوتا ہے
- `RequestInfoMessage` - ٹائپ شدہ درخواست کے ڈیٹا کے لیے بنیادی کلاس
- `RequestResponse` - انسانی جوابات کو درخواستوں کے ساتھ جوڑتا ہے
- `WorkflowOutputEvent` - ورک فلو کے نتائج کا پتہ لگانے کے لیے ایونٹ


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## مرحلہ 2: ساختی نتائج کے لیے Pydantic ماڈلز کی وضاحت کریں

یہ ماڈلز وہ **اسکیمہ** متعین کرتے ہیں جو ایجنٹس واپس کریں گے۔ ہم مشروط ورک فلو کے تمام ماڈلز کو برقرار رکھتے ہیں اور شامل کرتے ہیں:

**انسانی مداخلت کے لیے نیا:**
- `HumanFeedbackRequest` - `RequestInfoMessage` کا ذیلی کلاس جو انسانوں کو بھیجے جانے والے درخواست کے مواد کو متعین کرتا ہے
  - اس میں `prompt` (پوچھنے کے لیے سوال) اور `destination` (غیر دستیاب شہر کے بارے میں سیاق و سباق) شامل ہیں


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## مرحلہ 3: ہوٹل بکنگ کا ٹول بنائیں

وہی ٹول جو مشروط ورک فلو سے ہے - یہ چیک کرتا ہے کہ منزل میں کمرے دستیاب ہیں یا نہیں۔


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## مرحلہ 4: روٹنگ کے لیے کنڈیشن فنکشنز کی تعریف کریں

ہمیں اپنے انسانی مداخلت والے ورک فلو کے لیے **چار کنڈیشن فنکشنز** کی ضرورت ہے:

**مشروط ورک فلو سے:**
1. `has_availability_condition` - جب ہوٹل دستیاب ہوں تو روٹ کرتا ہے
2. `no_availability_condition` - جب ہوٹل دستیاب نہ ہوں تو روٹ کرتا ہے

**انسانی مداخلت کے لیے نیا:**
3. `user_wants_alternatives_condition` - جب صارف متبادل کے لیے "ہاں" کہے تو روٹ کرتا ہے
4. `user_declines_alternatives_condition` - جب صارف متبادل کے لیے "نہیں" کہے تو روٹ کرتا ہے


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## مرحلہ 5: فیصلہ مینیجر ایگزیکیوٹر بنائیں

یہ **انسانی مداخلت کے نمونے کا مرکزی حصہ** ہے! `DecisionManager` ایک حسب ضرورت `Executor` ہے جو:

1. **انسانی رائے حاصل کرتا ہے** `RequestResponse` اشیاء کے ذریعے
2. **صارف کے فیصلے کو پروسیس کرتا ہے** (ہاں/نہیں)
3. **ورک فلو کو راستہ دیتا ہے** مناسب ایجنٹس کو پیغامات بھیج کر

اہم خصوصیات:
- `@handler` ڈیکوریٹر استعمال کرتا ہے تاکہ طریقوں کو ورک فلو کے مراحل کے طور پر ظاہر کیا جا سکے
- `RequestResponse[HumanFeedbackRequest, str]` وصول کرتا ہے جس میں اصل درخواست اور صارف کا جواب دونوں شامل ہوتے ہیں
- سادہ "ہاں" یا "نہیں" پیغامات دیتا ہے جو ہماری شرطی افعال کو متحرک کرتے ہیں


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## مرحلہ 6: کسٹم ڈسپلے ایگزیکیوٹر بنائیں

وہی ڈسپلے ایگزیکیوٹر جو کنڈیشنل ورک فلو سے ہے - حتمی نتائج کو ورک فلو آؤٹ پٹ کے طور پر فراہم کرتا ہے۔


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## مرحلہ 7: ماحول کے متغیرات لوڈ کریں

LLM کلائنٹ کو ترتیب دیں (GitHub Models، Azure OpenAI، یا OpenAI)۔


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## مرحلہ 8: AI ایجنٹس اور ایگزیکیوٹرز بنائیں

ہم **چھ ورک فلو اجزاء** بناتے ہیں:

**ایجنٹس (AgentExecutor میں لپیٹے ہوئے):**
1. **availability_agent** - ٹول کے ذریعے ہوٹل کی دستیابی چیک کرتا ہے
2. **confirmation_agent** - 🆕 انسانی تصدیق کی درخواست تیار کرتا ہے
3. **alternative_agent** - متبادل شہروں کی تجویز دیتا ہے (جب صارف ہاں کہے)
4. **booking_agent** - بکنگ کی حوصلہ افزائی کرتا ہے (جب کمرے دستیاب ہوں)
5. **cancellation_agent** - 🆕 منسوخی کا پیغام سنبھالتا ہے (جب صارف نہ کہے)

**خصوصی ایگزیکیوٹرز:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor` جو انسانی ان پٹ کے لیے ورک فلو کو روکتا ہے
7. **decision_manager** - 🆕 حسب ضرورت ایگزیکیوٹر جو انسانی جواب کی بنیاد پر راستہ طے کرتا ہے (پہلے ہی اوپر بیان کیا گیا ہے)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## مرحلہ 9: انسانی مداخلت کے ساتھ ورک فلو بنائیں

اب ہم ورک فلو گراف کو **مشروط راستہ بندی** کے ساتھ تشکیل دیتے ہیں، جس میں انسانی مداخلت کا راستہ شامل ہے:

**ورک فلو کا ڈھانچہ:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**اہم راستے:**
- `availability_agent → confirmation_agent` (جب کمرے دستیاب نہ ہوں)
- `confirmation_agent → prepare_human_request` (قسم تبدیل کریں)
- `prepare_human_request → request_info_executor` (انسانی مداخلت کے لیے توقف)
- `request_info_executor → decision_manager` (ہمیشہ - RequestResponse فراہم کرتا ہے)
- `decision_manager → alternative_agent` (جب صارف "ہاں" کہے)
- `decision_manager → cancellation_agent` (جب صارف "نہیں" کہے)
- `availability_agent → booking_agent` (جب کمرے دستیاب ہوں)
- تمام راستے `display_result` پر ختم ہوتے ہیں


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## مرحلہ 10: ٹیسٹ کیس 1 چلائیں - شہر بغیر دستیابی کے (پیرس انسانی تصدیق کے ساتھ)

یہ ٹیسٹ **انسانی مداخلت کے مکمل عمل** کو ظاہر کرتا ہے:

1. پیرس میں ہوٹل کی درخواست کریں
2. availability_agent چیک کرتا ہے → کوئی کمرے دستیاب نہیں
3. confirmation_agent انسانی سوال تیار کرتا ہے
4. request_info_executor **ورک فلو کو روکتا ہے** اور `RequestInfoEvent` جاری کرتا ہے
5. **ایپلیکیشن ایونٹ کا پتہ لگاتی ہے اور کنسول میں صارف کو اشارہ دیتی ہے**
6. صارف "ہاں" یا "نہیں" ٹائپ کرتا ہے
7. ایپلیکیشن جواب `send_responses_streaming()` کے ذریعے بھیجتی ہے
8. decision_manager جواب کی بنیاد پر راستہ منتخب کرتا ہے
9. حتمی نتیجہ دکھایا جاتا ہے

**اہم پیٹرن:**
- پہلے دور کے لیے `workflow.run_stream()` استعمال کریں
- اگلے دور کے لیے `workflow.send_responses_streaming(pending_responses)` استعمال کریں
- جب انسانی ان پٹ کی ضرورت ہو تو `RequestInfoEvent` سنیں
- حتمی نتائج حاصل کرنے کے لیے `WorkflowOutputEvent` سنیں


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## مرحلہ 11: ٹیسٹ کیس 2 چلائیں - شہر کے ساتھ دستیابی (اسٹاک ہوم - انسانی ان پٹ کی ضرورت نہیں)

یہ ٹیسٹ **براہ راست راستہ** دکھاتا ہے جب کمرے دستیاب ہوں:

1. اسٹاک ہوم میں ہوٹل کی درخواست کریں  
2. availability_agent چیک کرتا ہے → کمرے دستیاب ✅  
3. booking_agent بکنگ کی تجویز دیتا ہے  
4. display_result تصدیق دکھاتا ہے  
5. **انسانی ان پٹ کی ضرورت نہیں!**

جب کمرے دستیاب ہوں تو ورک فلو مکمل طور پر انسانی ان پٹ کے راستے کو نظرانداز کر دیتا ہے۔


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## اہم نکات اور انسانی مداخلت کے بہترین طریقے

### ✅ آپ نے کیا سیکھا:

#### 1. **RequestInfoExecutor پیٹرن**
Microsoft Agent Framework میں انسانی مداخلت کے پیٹرن میں تین اہم اجزاء شامل ہیں:
- `RequestInfoExecutor` - ورک فلو کو روک کر ایونٹس جاری کرتا ہے
- `RequestInfoMessage` - ٹائپڈ درخواست کے پیلوڈز کے لیے بنیادی کلاس (اسے سب کلاس کریں!)
- `RequestResponse` - انسانی جوابات کو اصل درخواستوں کے ساتھ جوڑتا ہے

**اہم سمجھ:**
- `RequestInfoExecutor` خود ان پٹ جمع نہیں کرتا - یہ صرف ورک فلو کو روکتا ہے
- آپ کے ایپلیکیشن کوڈ کو `RequestInfoEvent` سننا ہوگا اور ان پٹ جمع کرنا ہوگا
- آپ کو `send_responses_streaming()` کو ایک dict کے ساتھ کال کرنا ہوگا جو `request_id` کو صارف کے جواب کے ساتھ میپ کرتا ہے

#### 2. **اسٹریمنگ ایگزیکیوشن پیٹرن**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **ایونٹ ڈرائیو آرکیٹیکچر**
ورک فلو کو کنٹرول کرنے کے لیے مخصوص ایونٹس سنیں:
- `RequestInfoEvent` - انسانی ان پٹ کی ضرورت ہے (ورک فلو روکا ہوا)
- `WorkflowOutputEvent` - حتمی نتیجہ دستیاب ہے (ورک فلو مکمل)
- `WorkflowStatusEvent` - اسٹیٹ میں تبدیلیاں (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS, وغیرہ)

#### 4. **@handler کے ساتھ کسٹم ایگزیکیوٹرز**
`DecisionManager` دکھاتا ہے کہ ایگزیکیوٹرز کیسے بنائیں جو:
- `@handler` ڈیکوریٹر استعمال کرتے ہیں تاکہ میتھڈز کو ورک فلو اسٹیپس کے طور پر ظاہر کریں
- ٹائپڈ میسیجز وصول کریں (جیسے `RequestResponse[HumanFeedbackRequest, str]`)
- ورک فلو کو دوسرے ایگزیکیوٹرز کو میسیجز بھیج کر روٹ کریں
- `WorkflowContext` کے ذریعے کانٹیکسٹ تک رسائی حاصل کریں

#### 5. **انسانی فیصلوں کے ساتھ مشروط روٹنگ**
آپ شرطی فنکشنز بنا سکتے ہیں جو انسانی جوابات کا جائزہ لیتے ہیں:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 حقیقی دنیا کی ایپلیکیشنز:

1. **منظوری کے ورک فلو**
   - خرچ کی رپورٹس پر کارروائی کرنے سے پہلے مینیجر کی منظوری حاصل کریں
   - خودکار ای میلز بھیجنے سے پہلے انسانی جائزہ کی ضرورت ہو
   - اعلیٰ قدر کے لین دین کو انجام دینے سے پہلے تصدیق کریں

2. **مواد کی نگرانی**
   - مشکوک مواد کو انسانی جائزہ کے لیے فلیگ کریں
   - ایج کیسز پر حتمی فیصلہ کرنے کے لیے موڈریٹرز سے پوچھیں
   - جب AI کا اعتماد کم ہو تو انسانوں کو بڑھائیں

3. **کسٹمر سروس**
   - AI کو معمولی سوالات خودکار طور پر ہینڈل کرنے دیں
   - پیچیدہ مسائل کو انسانی ایجنٹس تک بڑھائیں
   - صارف سے پوچھیں کہ کیا وہ انسان سے بات کرنا چاہتے ہیں

4. **ڈیٹا پروسیسنگ**
   - انسانوں سے مبہم ڈیٹا انٹریز کو حل کرنے کے لیے پوچھیں
   - غیر واضح دستاویزات کی AI تشریحات کی تصدیق کریں
   - صارفین کو متعدد درست تشریحات میں سے انتخاب کرنے دیں

5. **حفاظتی اہمیت کے نظام**
   - ناقابل واپسی اقدامات سے پہلے انسانی تصدیق کی ضرورت ہو
   - حساس ڈیٹا تک رسائی سے پہلے منظوری حاصل کریں
   - ریگولیٹڈ انڈسٹریز (صحت، مالیات) میں فیصلوں کی تصدیق کریں

6. **انٹرایکٹو ایجنٹس**
   - بات چیت کرنے والے بوٹس بنائیں جو فالو اپ سوالات پوچھتے ہیں
   - وزرڈز بنائیں جو صارفین کو پیچیدہ عمل کے ذریعے رہنمائی کرتے ہیں
   - ایجنٹس ڈیزائن کریں جو انسانوں کے ساتھ قدم بہ قدم تعاون کرتے ہیں

### 🔄 موازنہ: انسانی مداخلت کے ساتھ اور بغیر

| خصوصیت | مشروط ورک فلو | انسانی مداخلت کے ساتھ ورک فلو |
|---------|---------------------|---------------------------|
| **ایگزیکیوشن** | واحد `workflow.run()` | لوپ کے ساتھ `run_stream()` + `send_responses_streaming()` |
| **صارف ان پٹ** | کوئی نہیں (مکمل طور پر خودکار) | انٹرایکٹو پرامپٹس `input()` یا UI کے ذریعے |
| **اجزاء** | ایجنٹس + ایگزیکیوٹرز | + RequestInfoExecutor + DecisionManager |
| **ایونٹس** | صرف AgentExecutorResponse | RequestInfoEvent, WorkflowOutputEvent, وغیرہ |
| **روکنا** | کوئی روکنا نہیں | ورک فلو RequestInfoExecutor پر رک جاتا ہے |
| **انسانی کنٹرول** | کوئی انسانی کنٹرول نہیں | انسان اہم فیصلے کرتے ہیں |
| **استعمال کا کیس** | آٹومیشن | تعاون اور نگرانی |

### 🚀 جدید پیٹرن:

#### متعدد انسانی فیصلہ پوائنٹس
آپ ایک ہی ورک فلو میں متعدد `RequestInfoExecutor` نوڈز رکھ سکتے ہیں:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### ٹائم آؤٹ ہینڈلنگ
انسانی جوابات کے لیے ٹائم آؤٹ نافذ کریں:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### رچ UI انٹیگریشن
`input()` کے بجائے، ویب UI، Slack، Teams وغیرہ کے ساتھ انٹیگریٹ کریں:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### مشروط انسانی مداخلت
صرف مخصوص حالات میں انسانی ان پٹ کے لیے پوچھیں:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ بہترین طریقے:

1. **ہمیشہ RequestInfoMessage کو سب کلاس کریں**
   - ٹائپ سیفٹی اور ویلیڈیشن فراہم کرتا ہے
   - UI رینڈرنگ کے لیے رچ کانٹیکسٹ کو فعال کرتا ہے
   - ہر درخواست کی قسم کے ارادے کو واضح کرتا ہے

2. **وضاحتی پرامپٹس استعمال کریں**
   - آپ جو پوچھ رہے ہیں اس کے بارے میں کانٹیکسٹ شامل کریں
   - ہر انتخاب کے نتائج کی وضاحت کریں
   - سوالات کو سادہ اور واضح رکھیں

3. **غیر متوقع ان پٹ کو ہینڈل کریں**
   - صارف کے جوابات کی توثیق کریں
   - غلط ان پٹ کے لیے ڈیفالٹس فراہم کریں
   - واضح ایرر میسیجز دیں

4. **درخواست IDs کو ٹریک کریں**
   - request_id اور جوابات کے درمیان تعلق کا استعمال کریں
   - اسٹیٹ کو دستی طور پر منظم کرنے کی کوشش نہ کریں

5. **نان بلاکنگ کے لیے ڈیزائن کریں**
   - ان پٹ کے انتظار میں تھریڈز کو بلاک نہ کریں
   - پورے میں async پیٹرن استعمال کریں
   - متوازی ورک فلو انسٹینسز کو سپورٹ کریں

### 📚 متعلقہ تصورات:

- **ایجنٹ مڈل ویئر** - ایجنٹ کالز کو انٹرسیپٹ کریں (پچھلا نوٹ بک)
- **ورک فلو اسٹیٹ مینجمنٹ** - ورک فلو اسٹیٹ کو رنز کے درمیان برقرار رکھیں
- **ملٹی ایجنٹ تعاون** - انسانی مداخلت کو ایجنٹ ٹیموں کے ساتھ جوڑیں
- **ایونٹ ڈرائیو آرکیٹیکچرز** - ایونٹس کے ساتھ ری ایکٹیو سسٹمز بنائیں

---

### 🎓 مبارک ہو!

آپ نے Microsoft Agent Framework کے ساتھ انسانی مداخلت والے ورک فلو کو مہارت سے سیکھ لیا ہے! آپ اب جانتے ہیں کہ:
- ✅ ورک فلو کو انسانی ان پٹ جمع کرنے کے لیے روکیں
- ✅ RequestInfoExecutor اور RequestInfoMessage استعمال کریں
- ✅ ایونٹس کے ساتھ اسٹریمنگ ایگزیکیوشن کو ہینڈل کریں
- ✅ @handler کے ساتھ کسٹم ایگزیکیوٹرز بنائیں
- ✅ انسانی فیصلوں کی بنیاد پر ورک فلو کو روٹ کریں
- ✅ انٹرایکٹو AI ایجنٹس بنائیں جو انسانوں کے ساتھ تعاون کرتے ہیں

**یہ قابل اعتماد اور کنٹرول ایبل AI سسٹمز بنانے کے لیے ایک اہم پیٹرن ہے!** 🚀



---

**ڈسکلیمر**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کا استعمال کرتے ہوئے ترجمہ کی گئی ہے۔ ہم درستگی کے لیے کوشش کرتے ہیں، لیکن براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا غیر درستیاں ہو سکتی ہیں۔ اصل دستاویز کو اس کی اصل زبان میں مستند ذریعہ سمجھا جانا چاہیے۔ اہم معلومات کے لیے، پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ ہم اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کے ذمہ دار نہیں ہیں۔
